In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from numpy import mean
from numpy import std
from mlxtend.evaluate import paired_ttest_5x2cv,combined_ftest_5x2cv
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import IsolationForest

# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Covariance=['spherical', 'tied', 'diag', 'full']
Params=['kmeans', 'random']
skf = StratifiedKFold(n_splits=2) 
fold=0
test=False
for train_index, test_index in skf.split(X, y):
        fold=fold+1
        k = 0
        liste =[0]*8
        
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        for i in Covariance:
                for j in Params:
                       
                       
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i)
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.1:
    
                                y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                        
                        F1=round(f1_score(y_test,y_pred),3)
                        recall=round(recall_score(y_test,y_pred),3)
                        precision=round(precision_score(y_test,y_pred,zero_division=0),3)
                        accuracy=round(accuracy_score(y_test,y_pred),3)
                        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred)
                        roc_auc = metrics.auc(fpr, tpr)
                        if test==False :
                                data = np.array([[i,j,F1,recall,precision,accuracy,roc_auc,fold]])
                                test=True
                        else :
                                data = np.append(data, [[i,j,F1,recall,precision,accuracy,roc_auc,fold]],axis=0)
                        liste[k]=gmm
                        k = k+1
                       


              
                        
             


P-value: 0.313, t-Statistic: -1.122
P-value: 0.978, t-Statistic: 0.029
P-value: 0.195, t-Statistic: -1.496
P-value: 0.227, t-Statistic: 1.376
P-value: 1.000, t-Statistic: 0.000
P-value: 0.288, t-Statistic: -1.188
P-value: 0.134, t-Statistic: 1.788
P-value: 1.000, t-Statistic: 0.000
P-value: 0.834, t-Statistic: 0.221
P-value: 0.202, t-Statistic: 1.468
P-value: 0.332, t-Statistic: 1.073
P-value: 0.363, t-Statistic: 1.001
P-value: 0.940, t-Statistic: 0.080
P-value: 0.067, t-Statistic: -2.336


In [129]:
import pandas as pd
from scipy.stats import mannwhitneyu
from statsmodels.formula.api import ols

df = pd.DataFrame(data=data,columns=['Covariance','Parametre','F1','recall','precision','accuracy','AUC','numero_fold'])
df


,Covariance,Parametre,F1,recall,precision,accuracy,AUC,numero_fold
0,spherical,kmeans,0.905,0.992,0.832,0.927,0.9417792792792793,1
1,spherical,random,0.905,0.992,0.832,0.927,0.9417792792792793,1
2,tied,kmeans,0.905,0.875,0.938,0.936,0.9217342342342342,1
3,tied,random,0.542,0.833,0.402,0.506,0.5810810810810811,1
4,diag,kmeans,0.862,0.992,0.763,0.889,0.9125000000000001,1
5,diag,random,0.862,0.992,0.763,0.889,0.9125000000000001,1
6,full,kmeans,0.811,1.0,0.682,0.836,0.8738738738738738,1
7,full,random,0.8,1.0,0.667,0.825,0.8648648648648649,1
8,spherical,kmeans,0.948,1.0,0.902,0.962,0.9707207207207208,2
9,spherical,random,0.975,0.966,0.983,0.982,0.9786887728064199,2


In [130]:

estimateur1 = GaussianMixture(n_components=2, covariance_type = 'full', init_params='kmeans')
estimateur2 = KNeighborsClassifier(n_neighbors=11,metric='euclidean')

y=np.ravel(y)
t,p = paired_ttest_5x2cv(estimator1=estimateur1,estimator2=estimateur2,X=X, y=y, scoring='f1', random_seed=1)
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))
t,p = combined_ftest_5x2cv(estimator1=estimateur1,estimator2=estimateur2,X=X, y=y, scoring='f1', random_seed=1)
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))



P-value: 0.061, t-Statistic: -2.413
P-value: 0.217, t-Statistic: 2.079
